In [ ]:
import pandas as pd
import os as os

In [ ]:
# To force change - cleared outputs. 

In [ ]:
def check_file_exists() -> str:
    while True:
        filename = input("Enter filename. Relative paths are acceptable ... \n")
        if os.path.isfile(filename):
            print("File Exists. OK to continue")
            return filename
        else:
            print(f"File {filename} does not exist. Check path and filename.")

# Get the file name and read into a df if exists
xls = check_file_exists()
excel_map = pd.read_excel(xls, sheet_name=None)

# Clean the data of trailing spaces for the Categories column 
for sheet, df in excel_map.items():
    if 'Category' in df.columns:
        # Strip trailing spaces from the 'Category' column values
        df['Category'] = df['Category'].str.rstrip()

In [ ]:
# Open up excel into an object
# This takes a long time - in production might use different approach
# Was taking a long time due to capacity blank cells pasted into worksheets!
#xls = pd.ExcelFile('tyb2122.xlsx')

# Read sheets into a dictionary of sheets
#excel_map = pd.read_excel(xls, sheet_name=None)
#print(excel_map.keys())

In [ ]:
# Copy into a dict just because of naming below. No real need to do this apart from renaming everything below
df_dict = excel_map
print(type(df_dict))

In [ ]:
print(df_dict['Aug 2024'])

In [ ]:
# Create a list of sheets to ignore
# Iterate over the sheets in the dict created above (misnamed stuff when starting out is all)
# Exclude the sheet if it's in the list
# Do some DQ analysis
# Otherwise - Group the data in the current sheet. Convert in-line to a dataframe
# Concatenate these as we go along
# Group the final sheet up to category level as it is the complete set

exclude_sheets = ['Template', 'Lists']

month_totals_df= pd.DataFrame()
totals_df= pd.DataFrame()

for k in df_dict.keys():
    #print(f"{k =}")
    if k in exclude_sheets:
        #print("Excluded")
        continue
    else:
        
        # Check actuals column for nulls:
        check_nan = df_dict[k]['Actual'].isnull().values.any()
        if check_nan:
            print(f'Null Actuals in sheet: {k}')
        try:
            current_month_df = pd.DataFrame(df_dict[k].groupby('Category')['Actual'].sum())
        except:
            print(f'Error grouping: {k}')
        #Add current month to new dataframe
        current_month_df['month'] = k
        #display(current_month_df.head(10))
        #display(totals_df)
        month_totals_df = pd.concat([month_totals_df, current_month_df], axis=0) #, ignore_index=False)

# Group the months
totals_df = pd.DataFrame(month_totals_df.groupby('Category')['Actual'].sum()).reset_index()
#print(totals_df)
rental_categories = ['Airbnb', 'Cottages', 'Private']



In [ ]:
# Format floats to 2 places:
month_totals_df.style.format({"Actual" : ".2f"})
totals_df.style.format({"Actual" : ".2f"})
# Output to csv
month_totals_df.to_csv('categories_all_months.csv', float_format='%.2f')

totals_df.to_csv('category_totals.csv', float_format='%.2f')

In [ ]:
# Format floats to 2 places:
# month_totals_df.style.format({"Actual" : "{.2f}"})
month_totals_df.style.format({"Actual": "{:.2f}".format})
#totals_df.style.format({"Actual": "{.2f}"})
totals_df.style.format({"Actual": "{:.2f}"})


In [ ]:
print(totals_df)

In [ ]:
print(f'month_totals: {month_totals_df.dtypes}')
print(f'totals: {totals_df.dtypes}')


In [ ]:
print(totals_df.head(100))


april_df = df['April 21']
print(april_df.head(5))

april_df.columns = ['Date', 'Category', 'Detail', 'In / Out', 'Amount', 'Actual', 'Notes',
       'Unnamed: 7', 'Unnamed: 8']
print(april_df.head(5))

In [ ]:
examine_df = df_dict['Mar 2025']
examine_df

In [ ]:
from IPython.display import clear_output

try:
    del df_dict["Template"]
    del df_dict["Lists"]
except Exception as e:
    print (e)
    
    
for month, month_df in df_dict.items():
    clear_output(wait=True)
    print(f"Displaying - {month}")
    #month_df = df_dict['January']
    display(month_df)
    #per_month_actuals = month_df.groupby('Category')['Actual'].sum()
    #per_month_actuals.head(100)
    input("Press ENTER to see next month")
    


In [ ]:
# TODO Per month


In [ ]:
month_totals_df.loc['Household']

In [ ]:
# WHERE CLAUSE ?
# Per Category
#display(totals_df)
where_df = month_totals_df.where(month_totals_df['month']=='Apr 2024')
display(where_df)


In [ ]:
# Source - https://stackoverflow.com/a
# Posted by starriet 차주녕
# Retrieved 2026-01-18, License - CC BY-SA 4.0
pd.set_option('display.min_rows', 500)  # <-add this!
pd.set_option('display.max_rows', 500)

pd.options.display.max_rows

month_totals_df.head(500)